In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

<img src="./img/monitoring.jpg"/>

#### setup the environment 

In [ ]:
import pandas as pd
from arcgis.gis import GIS
import arcgis

#### Create the GIS Connection

In [ ]:
gis = GIS(profile="your_dev_enterprise_profile")
user = gis.users.me
um = gis.users

### Managing Named User Licenses and Entitlements

In [ ]:
# list all licenses in the organization
license_list = gis.admin.license.all()

In [ ]:
license_list

In [ ]:
user = gis.users.search("creator")[0]
user

In [ ]:
l = license_list[1]
l.check(user.username)

In [ ]:
l.assign(user.username, ['runtimeanalysis'])

In [ ]:
l.plot()

In [ ]:
pro_license = gis.admin.license.get('ArcGIS Pro')
pro_license

In [ ]:
#get all users licensed for ArcGIS Pro
[prov['username'] for prov in pro_license.all()]

In [ ]:
pro_license.assign(user, ['desktopBasicN'])

In [ ]:
%matplotlib inline
pro_license.plot()

In [ ]:
# Get license object's report property
pro_license.report

In [ ]:
pro_license.report.iloc[8]["Users"]

### Querying and Assigning Named Licenses

In [ ]:
pro_license.user_entitlement('creator2')

To revoke an app's license from a user, call the revoke() method from the corresponding License object.

In [ ]:
pro_license.revoke(username='creator1', entitlements="*")

In [ ]:
pro_license.user_entitlement('creator1')

You can assign licenses to an application and its extensions using the assign() method.

In [ ]:
pro_license.assign(username='creator1', entitlements=['3DAnalystN',
  'bathymetryN',
  'businessStdN',
  'dataInteropN',
  'dataReviewerN',
  'desktopAdvN',
  'geostatAnalystN',
  'imageAnalystN',
  'networkAnalystN',
  'publisherN',
  'spatialAnalystN'])

In [ ]:
pro_license.user_entitlement('creator1')

In [ ]:
pro_license.revoke(username='creator1', entitlements="*")

# Generating Reports for Organization
Generate the reports of the overall usage of the organizations.  
Reports define organization usage metrics in one place for the day, week, or month.  
Administrators can monitor who is using which services, consuming how much credits and storage within certain time period.



In [ ]:
import datetime as _dt
import pandas as pd

In [ ]:
date_time_str = '06/18/23'
then = _dt.datetime.strptime(date_time_str, '%m/%d/%y')
val = int(then.timestamp() * 1000)

In [ ]:
gis.users.me.report?

In [ ]:
# Create the report, this will create an item in your organization
final_item = gis.users.me.report(
            report_type='content', 
    duration="weekly", start_time=val
        )
final_item

In [ ]:
# Read the item data into a temporary CSV file
csv_file = final_item.get_data()

# Read the CSV as a DataFrame
df = pd.read_csv(csv_file)

In [ ]:
df.head()

#### See the Most Viewed Items

In [ ]:
df.sort_values(by=['View Counts'], ascending=False).head()

#### See the Largest Sized Items

In [ ]:
df.sort_values(by=['File Storage Size'], ascending=False).head()[['Title','Item Type', 'File Storage Size']]

#### Understand Storage by User

In [ ]:
gb = df.groupby("Owner")['File Storage Size'].sum()
gb.nlargest(4).plot(kind='barh')

In [ ]:
gb

## Examine Login Activity

In [ ]:
date_time_str = '18/06/23'
then = _dt.datetime.strptime(date_time_str, '%d/%m/%y')
val = int(then.timestamp() * 1000)
user_report_item = gis.users.me.report(
            report_type='users', duration="weekly", start_time=val
        )
user_report_item

In [ ]:
# Read the item data into a temporary CSV file
csv_activity_file = user_report_item.get_data()

# Read the CSV as a DataFrame
df = pd.read_csv(csv_activity_file)

In [ ]:
q = df["# of Items Owned"] > 0
df[q]

## Using Search Operations to Monitor Sites

- The `search` and `advanced_search` operations allows administrators to query GIS systems easily to find out information about the system
- The information can easily be parsed using other 3rd party libraries like pandas
- This information can be done multiple time a day or hour

### Monitoring Content Using Searches

#### Content Searches

Example: Finding Content Types Generated Using Searches

Whole document on advanced searching: https://doc.arcgis.com/en/arcgis-online/reference/advanced-search.htm

In [ ]:
cm = gis.content

In [ ]:
import datetime as _dt
now = _dt.datetime.now()
then = now - _dt.timedelta(days=7)
search = cm.advanced_search(
            f"orgid: {gis.properties.id} AND created: [{int(then.timestamp()* 1000)} TO {int(now.timestamp()* 1000)}] AND accountid:{gis.properties.id}", 
            max_items=-1, as_dict=True)
pd.DataFrame(search['results']).type.value_counts()

#### User Searches

In [ ]:
import string
last_login = set()
users = gis.users.advanced_search(f"accountid:{gis.properties.id}", max_users=-1)
[user for user in users['results'] if user['lastLogin'] == -1]

**Find Users Haven't Logged in for 90+ Days**

In [ ]:
now = _dt.datetime.now(_dt.timezone.utc)
greater_than90days = [user for user in users['results'] if user['lastLogin'] > -1 and \
 (now - _dt.datetime.fromtimestamp(user['lastLogin']/1000, _dt.timezone.utc)).days >= 90]
greater_than90days

# Part 3 - Working with Backend Systems